### Interactive Playground: Provable Polytope Repair

In [1]:
import warnings; warnings.filterwarnings("ignore")
from tutorial_helpers import *

@interact_polytope_repair_with
def polytope_repair(N, x1, x2, lb, ub, ap):
    """ Given a DNN `N`, a 1-d input polytope [x1, x2] defined by the endpoints
        `x1` and `x2`, and the output bound [lb, ub], find new parameters `\theta'`
        for `N` such that `\forall x \in [x1, x2]. N(x; \theta') \in [lb, ub]`
        while minimizing the changes to `N`. 

    Args:
        N (nn.Module): the network to repair. 
        x1, x2 (float): endpoints of the 1-d input polytope to repair. 
        lb, ub (float): the output bound. 
        ap (np.ndarray[bool]): the activation pattern of ReLU layers.

    Returns:
        Repaired network `N` if feasible, otherwies `None`. 
    """
    endpoints = torch.tensor([x1, x2])[:,None]

    # Use Gurobi as the optimizer.
    optimizer = st.GurobiSolver().verbose_(False)

    # Attach `N` to the optimizer and enables the repair mode.
    N.to(optimizer).repair()

    # Make the first layer weight and all-layer's bias to be editable.
    N.requires_symbolic_weight_and_bias()

    # Encode the symbolic output for the input endpoints with the same activation pattern `ap`.
    symbolic_output = N(endpoints, pattern=ap)

    # Define and solve the LP problem.
    succeed =  optimizer.solve(
        # add the output constraints
        lb <= symbolic_output, symbolic_output <= ub,
        
        # specify the optimization objective 
        minimize = N.delta(endpoints)
    )

    # Check if feasible.
    if not succeed:
        print("Infeasible!")
        return None
    
    # Update `N` with the optimal parameters.
    N.update_().eval()

    # Return the repaired network.
    return N

Output(layout=Layout(height='800px'))